# Trabajar con entornos (environments)

In [ ]:
# mostrar info de azure-ai-ml

! pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


## Ejecutar un script como job

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

Después de crear el script, puede ejecutar el script como un trabajo.El script utiliza bibliotecas comunes.Por lo tanto, puede usar un entorno curado que incluya pandas, numpy y scikit-learn, entre otros.

El trabajo utiliza la última versión del entorno curado: `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="<vm o cluster>",
    display_name="diabetes-train-curated-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Listar environments

In [ ]:
# listar los entornos

# para enumerar los entornos usando el SDK de Python:

envs = ml_client.environments.list()
for env in envs:
    print(env.name)



In [ ]:
# para revisar los detalles de un entorno específico, puede recuperar el entorno por su nombre registrado:

env = ml_client.environments.get(name="AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu", version="1")
print(env)

Tener en cuenta que todos los entornos curados tienen nombres que comienzan **AzureML-** (No se puede usar este prefijo para sus propios entornos).

Para revisar un entorno específico, puede recuperar un entorno por su nombre y versión.Por ejemplo, puede recuperar la *descripción* y *etiquetas* del entorno curado que utilizó para el trabajo anterior:

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env.description, env.tags)

## Crear y usar un entorno personalizado

- Si un entorno curado no incluye todos los paquetes de Python que necesita para ejecutar tu script, se puede crear un entorno personalizado.
- Al enumerar todos los paquetes necesarios en un entorno, se puede volver a ejecutar fácilmente los scripts.
- Todas las dependencias se almacenan en el entorno que luego se puede especificar en la configuración del job, independientemente del cómputo que se utilice.

Por ejemplo, se puede crear un entorno a partir de una imagen de Docker.

In [ ]:
# Crear entorno
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/curated/sklearn-1.5:26",
    name="sklearn-env-ric",
    description="Environment created from a Azure ML Docker image.",
    version="3"
)

ml_client.environments.create_or_update(env_docker_image)

El entorno ahora está registrado en el espacio de trabajo y se puede hacer referencia a él cuando ejecuta un script como un job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="/home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src",
    command="python diabetes-training.py",
    environment="sklearn-env-ric:3",
    compute="ricardoenm",
    display_name="diabetes-train-custom-env-ric",
    experiment_name="diabetes-train-custom-env-ric"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitorear el job en", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> ¡El job fallará rápidamente!Revise el mensaje de error. </p>

El mensaje de error le dirá que no hay un módulo llamado pandas.Hay dos causas posibles para tal error:

- El script usa pandas pero no importó la librería (`import pandas as pd`). 
- El script importa la librería en la parte superior del script, pero el proceso no tenía la librería instalada (`pip install pandas`).

Crearemos un entorno nuevo, utilizando la imagen base Docker utilizada en el trabajo anterior.
- Generemos una especificación para asegurarnos de que se instalarán los paquetes necesarios.

In [ ]:
# Añadir especificaciones al entorno
%%writefile /home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src/conda-env.yml

name: basic-env-cpu
channels:
    - conda-forge
dependencies:
    - python=3.11
    - scikit-learn
    - pandas
    - numpy
    - matplotlib
    pip:
        - mltable

Tener en cuenta que todas las dependencias necesarias se incluyen en el archivo de especificación de Conda para que el script se ejecute correctamente.

- Crear un nuevo entorno utilizando la imagen base Docker **y** el archivo de especificación de conda para agregar las dependencias necesarias.
- Azure Machine Learning construirá el entorno Conda sobre la imagen de Docker definida.

In [22]:
# Crear nuevo entorno
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/curated/sklearn-1.5:26",
    conda_file="/home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src/conda-env.yml",
    name="sklearn-env-ric",
    description="Environment created from a Azure ML Docker image.",
    version="4"
)

ml_client.environments.create_or_update(env_docker_image)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/curated/sklearn-1.5:26', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'sklearn-env-ric', 'description': 'Environment created from a Azure ML Docker image.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml4/environments/sklearn-env-ric/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ricardoenm/code', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x73189d5c7970>, 'serialize': <msrest.serialization.Serializer object at 0x73189d5c60b0>, 'version': '4', 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.11', 'scikit-learn', 'pa

Ahora, se puede enviar un job con el nuevo entorno para ejecutar el script:

In [23]:
from azure.ai.ml import command

# configure job
job = command(
    code="/home/azureuser/cloudfiles/code/Users/formacion/azure-machine-learning/1.Introduccion_AML_Pipelines/labs/src",
    command="python diabetes-training.py",
    environment="sklearn-env-ric:4",
    compute="ricardoenm",
    display_name="diabetes-train-custom-env-ric",
    experiment_name="diabetes-train-custom-env-ric"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitorear el job en", aml_url)

Uploading src (0.52 MBs): 100%|██████████| 520449/520449 [00:00<00:00, 4195392.25it/s]




Monitorear el job en https://ml.azure.com/runs/maroon_flag_8t86vs8tf7?wsid=/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4&tid=8b85184d-f6f0-4ace-80dd-c3bda5a6c156
